In [1]:
import recommenders

from src.datasets import daocensus

%load_ext autoreload
%autoreload 2

In [2]:
SEED = 42
TOP_K = 5
EPOCHS = 50
BATCH_SIZE = 512

In [3]:
dfv, dfp = daocensus.get("./data/daos-census", 'Decentraland', 'snapshot')# , min_vpu=6)
print(dfv.info())
print(dfp.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116560 entries, 0 to 116559
Data columns (total 10 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   platform       116560 non-null  object        
 1   name           116560 non-null  object        
 2   id             116560 non-null  object        
 3   proposal       116560 non-null  category      
 4   deployment     116560 non-null  object        
 5   platform_vote  116560 non-null  object        
 6   voter          116560 non-null  category      
 7   date           116560 non-null  datetime64[ns]
 8   choice         116560 non-null  object        
 9   weight         116560 non-null  float64       
dtypes: category(2), datetime64[ns](1), float64(1), object(6)
memory usage: 7.9+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1942 entries, 0 to 1941
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype         
---  ------  

In [4]:
def to_microsoft(dfv):
    df = dfv[['voter', 'proposal', 'date']].rename(columns={
        'voter': 'userID',
        'proposal': 'itemID',
        'date': 'timestamp',
    })
    df['itemID'] = df['itemID'].astype('str')
    df['rating'] = 1
    return df

df = to_microsoft(dfv)
df

,userID,itemID,timestamp,rating
0,0xe7af1c70f8f089c4c3bd71999692c6c5a15d9e2a,b86aa059-3d31-5d41-a472-70962816f779,2021-12-17 12:28:01,1
1,0xc54a6c3778016b06cbd126ccc3b5bc06c5f666fb,b86aa059-3d31-5d41-a472-70962816f779,2021-12-17 02:16:23,1
2,0xd82d005e8f8d5385db40ba23884a5c967bb1e8af,b86aa059-3d31-5d41-a472-70962816f779,2021-12-17 00:38:22,1
3,0xf4c64db66ffb301985f5ecd85c8f3f9c02f2659d,b86aa059-3d31-5d41-a472-70962816f779,2021-12-16 18:47:08,1
4,0xd5e9ef1cedad0d135d543d286a2c190b16cbb89e,b86aa059-3d31-5d41-a472-70962816f779,2021-12-16 18:32:15,1
...,...,...,...,...
116555,0x1156bf625b37623a86d004e66e0a01ec4b17e051,d083109e-4819-54b9-a01c-67bd5a770f65,2022-09-06 18:47:54,1
116556,0xbd03add5da0e173c67c9c1073ffba017147c42d4,d083109e-4819-54b9-a01c-67bd5a770f65,2022-09-06 18:44:04,1
116557,0x4da03f669dd9609dc49ca6f3451ba22d3b792395,d083109e-4819-54b9-a01c-67bd5a770f65,2022-09-06 18:21:14,1
116558,0xd6e62a97a55537cd04847bb73e22208bd20106aa,d083109e-4819-54b9-a01c-67bd5a770f65,2022-09-06 18:14:10,1


In [5]:
from recommenders.datasets.python_splitters import (
    python_stratified_split,
    python_chrono_split,
)

train, test = python_chrono_split(df, ratio=0.75)

In [6]:
from recommenders.models.deeprec.DataModel.ImplicitCF import ImplicitCF

data = ImplicitCF(train=train, test=test, seed=SEED)

/home/daviddavo/upm-tfm-notebooks/.direnv/python-3.9/lib/python3.9/site-packages/recommenders/models/deeprec/DataModel/ImplicitCF.py:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = train if test is None else train.append(test)


In [7]:
data.n_items, data.n_users

(1942, 7268)

In [8]:
from recommenders.models.deeprec.deeprec_utils import prepare_hparams

hparams = prepare_hparams(
    './config/lightgcn.yaml',
    n_layers=3,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    learning_rate=0.005,
    eval_epoch=5,
    top_k=TOP_K,
)

In [9]:
from recommenders.models.deeprec.models.graphrec.lightgcn import LightGCN

model = LightGCN(hparams, data, seed=SEED)

2023-10-25 17:04:29.657681: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-25 17:04:29.657702: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-25 17:04:29.657716: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Already create adjacency matrix.
Already normalize adjacency matrix.
Using xavier initialization.


In [10]:
model.fit()

Epoch 1 (train)0.6s: train loss = 0.31916 = (mf)0.31862 + (embed)0.00054
Epoch 2 (train)0.4s: train loss = 0.09870 = (mf)0.09731 + (embed)0.00139
Epoch 3 (train)0.4s: train loss = 0.07097 = (mf)0.06917 + (embed)0.00180
Epoch 4 (train)0.5s: train loss = 0.05632 = (mf)0.05424 + (embed)0.00209
Epoch 5 (train)0.5s + (eval)1.0s: train loss = 0.04648 = (mf)0.04418 + (embed)0.00230, recall = 0.12036, ndcg = 0.10808, precision = 0.06340, map = 0.07418
Epoch 6 (train)0.4s: train loss = 0.03993 = (mf)0.03743 + (embed)0.00249
Epoch 7 (train)0.4s: train loss = 0.03598 = (mf)0.03332 + (embed)0.00265
Epoch 8 (train)0.4s: train loss = 0.03262 = (mf)0.02982 + (embed)0.00280
Epoch 9 (train)0.5s: train loss = 0.03054 = (mf)0.02761 + (embed)0.00293
Epoch 10 (train)0.4s + (eval)0.2s: train loss = 0.02866 = (mf)0.02563 + (embed)0.00303, recall = 0.13383, ndcg = 0.12290, precision = 0.06785, map = 0.08840
Epoch 11 (train)0.3s: train loss = 0.02744 = (mf)0.02430 + (embed)0.00314
Epoch 12 (train)0.4s: train l

### Recommendation

We can call recommend_k_items to recommend k items for each user passed in this function. We set remove_seen=True to remove the items already seen by the user. The function returns a dataframe, containing each user and top k items recommended to them and the corresponding ranking scores.

In [11]:
topk_scores = model.recommend_k_items(test, top_k=TOP_K, remove_seen=True)
topk_scores.head()

,userID,itemID,prediction
0,0x3fdf6b7355c05ea6b3a224bb711cffbee8d20ac2,fc70266a-b8f4-5a35-82bb-f5f9764f5bec,11.026363
1,0x3fdf6b7355c05ea6b3a224bb711cffbee8d20ac2,55452f96-5839-53a4-ba2a-1b4fd5812796,8.571611
2,0x3fdf6b7355c05ea6b3a224bb711cffbee8d20ac2,78ea1f82-3ad1-59a5-a835-dd16ffb688be,8.283291
3,0x3fdf6b7355c05ea6b3a224bb711cffbee8d20ac2,9c65c60e-542b-55d9-90b8-c3975cb96569,8.049166
4,0x3fdf6b7355c05ea6b3a224bb711cffbee8d20ac2,72068dbe-a532-5d9c-bd73-ddbd3c40809f,7.852084


In [12]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28314 entries, 73107 to 110078
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   userID     28314 non-null  category      
 1   itemID     28314 non-null  object        
 2   timestamp  28314 non-null  datetime64[ns]
 3   rating     28314 non-null  int64         
dtypes: category(1), datetime64[ns](1), int64(1), object(1)
memory usage: 1.2+ MB


In [13]:
topk_scores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20930 entries, 0 to 20929
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   userID      20930 non-null  category
 1   itemID      20930 non-null  object  
 2   prediction  20930 non-null  float32 
dtypes: category(1), float32(1), object(1)
memory usage: 601.1+ KB


### 2.5.2 Evaluation
With topk_scores predicted by the model, we can evaluate how LightGCN performs on this test set.

In [14]:
from recommenders.evaluation.python_evaluation import map_at_k, ndcg_at_k, precision_at_k, recall_at_k

In [15]:
test['prediction'] = 1
eval_map = map_at_k(test, test, k=TOP_K)
eval_ndcg = ndcg_at_k(test, test, k=TOP_K)
eval_precision = precision_at_k(test, test, k=TOP_K)
eval_recall = recall_at_k(test, test, k=TOP_K)

print("Maximum achievable metrics")
print("MAP:\t%f" % eval_map,
      "NDCG:\t%f" % eval_ndcg,
      f"Precision@{TOP_K}:\t%f" % eval_precision,
      f"Recall@{TOP_K}:\t%f" % eval_recall, sep='\n')

Maximum achievable metrics
MAP:	0.851933
NDCG:	1.000000
Precision@5:	0.584138
Recall@5:	0.851933


In [16]:
eval_map = map_at_k(test, topk_scores, k=TOP_K)
eval_ndcg = ndcg_at_k(test, topk_scores, k=TOP_K)
eval_precision = precision_at_k(test, topk_scores, k=TOP_K)
eval_recall = recall_at_k(test, topk_scores, k=TOP_K)

print("MAP:\t%f" % eval_map,
      "NDCG:\t%f" % eval_ndcg,
      f"Precision@{TOP_K}:\t%f" % eval_precision,
      f"Recall@{TOP_K}:\t%f" % eval_recall, sep='\n')

MAP:	0.123398
NDCG:	0.164225
Precision@5:	0.090635
Recall@5:	0.175118


## 2.6 Infer embeddings

With infer_embedding method of LightGCN model, we can export the embeddings of users and items in the training set to CSV files for future use.

In [17]:
# model.infer_embedding(user_file, item_file)